# 第9章 ミドルウェア(Middleware)

## 9.1 概要

### ミドルウェア
- Djangoの主要な機能（モデル・テンプレート・ビュー）とWebサーバ/アプリケーションサーバの中間に位置する
- ビューを出入りする全てのリクエストとレスポンスをフックすることが出来る
- 有効にするミドルウェアは設定ファイル(settings.py)の`MIDDLEWARE`にリスト形式で列挙する
  - 記載した順番通りにミドルウェアが実行される
- 全てのミドルウェアがリクエストのたびに毎回実行される
  - 可能な限りデータベースへのアクセスは行わない
    - 必要な場合は`django.utils.functional.SimpleLazyObject`を使った遅延評価やキャッシュを使用する

## 9.2 主なミドルウェアの役割

### Djangoプロジェクトを作成したときにデフォルトで設定されるミドルウェア

~~~python
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware'
]
~~~


### 9.2.1 SecurityMiddleware
- リクエストおよびレスポンスへのセキュリティ強化を行う
- `HTTP`でアクセスした場合に`HTTPS`で始まるURLに自動的にリダイレクトするようにレスポンスを返すための機能がある（デフォルトではオフ）
  - 以下の設定を行うことで機能をオンに出来る
  ~~~python
SECURE_SSL_REDIRECT = True
  ~~~


### 9.2.2 SessionMiddleware
- セッションを有効にするためのミドルウェア
- 有効化する場合は設定ファイルの`INSTALLED_APPS`に`django.contrib.sessions`を入れておく必要がある
- デフォルトの設定ではセッションの格納先はデータベース（`django_session`データベース）になっている

#### セッションの使い方
~~~python
language = request.session.het('language')

if language is not None:
    request.session['_language'] = 'ja'
~~~

#### セッションの格納先をデータベースからキャッシュ（オンメモリ）に変更する
- パフォーマンス向上のために、セッションの格納先をデータベースからキャッシュ（オンメモリ）に変更することが出来る
  - この設定はキャッシュバックエンドに「Memcached」を利用している場合にのみするべき
~~~python
SESSION_ENGINE = 'django.contrib.sessions.backends.cache'
~~~

### 9.2.3 CommonMiddleware
- ユーザーエージェントによるアクセス制限やURLリライティングなど、いくつかの便利な機能を提供する
- このミドルウェアを導入すると、デフォルトでURL末尾に`/`を付与する機能が有効化される（`APPEND_SLASH`のデフォルト値が`True`のため）
  - リクエストされたURLがスラッシュで終わっておらず、URLconfにマッチするパターンが見つからない場合に、末尾に`/`を追加したURLに自動的にリダイレクトしてくれる
- URLの先頭に`www.`がない場合に`www.`で始まる同じURLにリダイレクトしてくれる機能はデフォルトでオフになっている
  - 必要な場合は`PREPEND_WWW`を`True`に設定する


### 9.2.4 CsrfViewMiddleware
- CSRFトークンを検証するために必要なミドルウェア
- POSTリクエストで送られてきたCSRFトークンの値と、Cookie（`CSRF_USE_SESSIONS`の設定次第ではセッション）に格納していた値を検証し、異なる場合は403エラーを返す


### 9.2.5 AuthenticationMiddleware
- リクエストのたびに、リクエストオブジェクトの`user`属性にユーザー情報をセットする
  - その際、ユーザーがログイン済みの場合にはセッションに保管されている`User`モデルのインスタンスを、未ログインの場合には`AnonymousUser`クラスのオブジェクトをセットする
- セッションを必要とするため、`SessionMiddleware`よりも後ろに設定する必要がある
  
#### 実装
~~~python
class AuthenticationMiddleware(MiddlewareMixin):
    def process_request(self, request):
        ...（略）...
        request.user = SimpleLazyObject(lambda; get_user(request))
~~~

`SimpleLazyObject`の遅延評価により、`get_user()`の処理は`request.user`にアクセスされて初めて動作するようになっている
→ リクエスト毎に重い処理が毎回実行されないような仕組みになっている


### 9.2.6 MessageMiddleware
- 画面に一度だけ出力する用途で使う、いわゆる「フラッシュメッセージ」の機能を提供するためのミドルウェア
  - この機構は「メッセージフレームワーク」と呼ばれる
  
### 9.2.7 XFrameOptionsMiddleware
- クリックジャッキング対策のためのミドルウェア
- `X-Frame-Options`ヘッダを全てのレスポンスに設定する

## 9.3 ミドルウェアの書き方

### ミドルウェアの実装
- `__init__()`と`__call__()`メソッドを持つ単純なクラスを実装することで、ミドルウェアを実装することが出来る

#### シンプルなミドルウェアクラスの例
~~~python
class SimpleMiddlware:
    def __init__(self, get_response):
        self.get_response = get_response
        
    def __call__(self, request):
        # リクエストへの前処理をここに記述
        
        response = self_get_response(request)
        
        # レスポンスへの後処理をここに記述
        
        return response
~~~


#### ちょっとしたアクセス制限をするためのミドルウェアの実装例（accounts/middleware.py）
~~~python
from django.core.exceptions import PermissionDenied
from django.urls import reverse

class SitePermissionMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response
        
    def __call__(self, request):
        # リクエストへの前処理をここに記述
        
        response = self.get_reponse(request)
        
        # レスポンスへの後処理をここに記述
        
        return response
    
    def process_view(self, request, view_func, view_args, view_kwargs):
        """ビューを呼び出す直前に呼び出される処理"""
        has_site_permission = False
        if request.user.is_superuser or request.user.is_staff:
            has_site_permission = True
            
        admin_index = reverse('admin:index')
        # 権限を持っていないユーザーが「/admin/」配下にアクセスしたら403エラー
        if request.path.startswith(admin_index):
            if not has_site_permission:
                raise PermissionDenied
                
        request.user_has_site_permission = has_site_permission
~~~

##### [django.urls.reverse()メソッド](https://docs.djangoproject.com/ja/2.2/ref/urlresolvers/#reverse)
- 引数で渡したURLパターン名からURLを逆引きする
  - URLパターン名は`urls.py`内の`urlpatterns`で、ルートからのURLパターン、対応するView関数、URLパターン名の組み合わせを設定する

## 9.4 ベストプラクティス8:メッセージフレームワークを使う

### フラッシュメッセージの表示
`base.html`に記述してテンプレート全体に適用しておくか、`include`タグでメッセージを出す可能性があるテンプレートに適用しておくと良い

### メッセージフレームワークを有効化する手順
*Djangoプロジェクトを`startproject`で初期構築したときにデフォルトで設定済になっている*
- 設定ファイル（`settings.py`）の`INSTALLED_APPS`に`django.contrib.messages`を追加
- 同じく`MIDDLEWARE`に以下を追加
  - `django.contrib.sessioms.middleware.SessionMiddleware`
  - `django.contrib.messages.middleware.MessageMiddleware`（`SessionMiddleware`より後に記載）
- 同じく`TEMPLATES.OPTIONS.context_processors`に`django.contrib.messages.context_processors.messages`を追加

### フラッシュメッセージの保存領域
- 初期設定では`django.contrib,messages.storage.fallback.FallbackStorage`
  - パフォーマンスのため、基本的にはCookieを使用し、Cookieで対応できないメッセージについてはセッションを使用する
  - 一部の場合にリダイレクトしたときにフラッシュメッセージが表示されない場合がある
- セッションにのみ保存する場合は、設定ファイルの`MESSAGE_STORAGE`の設定を変更する
  ~~~python
MESSAGE_STORAGE = `django.contrib.message.storage.session.SessionStorage`
  ~~~

### フラッシュメッセージの使用例
#### ビュー側の実装例（accounts/views.py）
~~~python
from django.contrib. import messages
from django.urls import reverse
from django.views import View

class LoginView(View):
    def post(self, request, *args, **kwargs):
        ...（略）...
        
        # フラッシュメッセージを画面に表示
        messages.info(request, "ログインしました。")
        
        return redirect(reverse('shop:index'))
~~~

#### テンプレート側の実装例（templates/\_message.html）
~~~html
{% if massages %}
<div class="ui relaxed divided list">
    {% for message in messages %}
    <div class="ui {% if messag.tags %}{{ message.tags }}{% endif %} message">
    {{ message }}</div>
    {% endfor %}
</div>
{% endif %}
~~~

### タグ
- フラッシュメッセージではメッセージ文字列に加えて「タグ」を付けることが出来る
- `extra_tags`キーワード引数に任意の文字列を渡すことで、`messages.tags`にタグが追加される

## 9.5 まとめ
- ミドルウェアは縁の下の力持ち
  - 普段何気なく使う機能（セッション系、セキュリティ系、フラッシュメッセージなど）が実はミドルウェアのおかげだったりする
- プロジェクト作成時に設定されるミドルウェアはとりあえずそのまま使うのが吉